<a href="https://colab.research.google.com/github/Midmost/Cloud_AI_mobile_ios/blob/main/AI/AI_imdb_0924.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow
from tensorflow.keras.datasets import imdb 

In [ ]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

#sample이라서 단어의 횟수로 비교하는 거지만 실제로는 단어의 가치에 중요도를 둬야해 욕한다거나...

실제 쇼핑몽의 챗봇에서는 욕을 하면 욕하지 말라고 하는 게 아니라 죄송하다고 사과해야함 그래야 유저가 물건을 구입하니까. 그러려면 유저가 욕하는지를 알아야겠지

In [ ]:
train_data.shape

In [ ]:
train_labels
#0은 negative
#1은 positive

In [ ]:
train_data[0:4] # 안에 리스트값들이 들어있고 그중 인덱스 0번의 들어있는 값들 

### 이 숫자들을 간단하게 복원해보자

In [ ]:
# word_index는 단어와 정수 인덱스를 매핑한 딕셔너리입니다
word_index = imdb.get_word_index()
# 정수 인덱스와 단어를 매핑하도록 뒤집습니다
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# 리뷰를 디코딩합니다.
# 0, 1, 2는 '패딩', '문서 시작', '사전에 없음'을 위한 인덱스이므로 3을 뺍니다
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

In [ ]:
decoded_review

## 데이터가 준비되었으니 이제 코드를 짜보자

In [ ]:
import numpy as np

## Vectorize

지금 단어에 대한 배열을 만들거야. 

하나의 배열이 있는데 인덱스가 0 ~9999 인 배열인데

전부 0으로 초기화되어 있는거야

여기에 단어가 들어올 때 마다 이 위치, 즉 몇 번째에 들어왔는지에 따라 0을 1로 바꿔주는거야


In [ ]:
def vectorize_sequences(sequences, dimension=10000): # dimension 다시 여쭤보자
  
  result = np.zeros((len(sequences), dimension)) # 0으로 채워진 배열을 만들어주자, 배열의 사이즈는 dimension (dimension만큼 채워지란 뜻)

  for i, sequence in enumerate(sequences):
    result[i, sequence] = 1

  return result

In [ ]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

#### 위 코드를 좀 더 풀어서 설명하자면

In [ ]:
np.zeros([2,10])

In [ ]:
np.zeros([2,10000])

In [ ]:
# 문장의 갯수를 확인하자
np.zeros((len(train_data), 10000)) # 밑으로는 25000개 옆으로는 10000개 짜리 배열이 생성!

In [ ]:
# enumerate 반복

#### 다시 돌아와서

In [ ]:
x_train[0] # 쭉 가다가 1이 보이네? 즉, 그자리에 해당하는 단어가 들어가있다는 뜻! 

원래는 영어 문장을 숫자로 바꿔버린거! => vectorizing 

In [ ]:
x_train[0].dtype

타입을 보니 float64네 이제 신경망을 만들고 학습을 해서 결과를 봐볼까?

In [ ]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32') 
# label 값이 저장되어있음

## 신경망 만들기

In [ ]:
# 신경망 모델 만들기
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy', # 둘 중 하나일 때는 binary! 헛갈리지마아아아
              metrics = ['accuracy']) 

model 완성! 이제 돌려볼건데 너무 많다 좀 자르자



In [ ]:
x_val = x_train[:10000] #학습용 데이터에서 만 개만 따로 먼저 떼어내서 만들어야 해
partial_x_train = x_train[10000:] # 만개부터 만 오천까지

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

이제 학습을 합시다

model.fit(partial_x_train, partial_y_train)
              feature값(x)과 label(y)값



In [ ]:
model.fit(partial_x_train, partial_y_train, epochs=3, batch_size=512)

훈련결과를 히스토리로 받자 

In [ ]:
history = model.fit(partial_x_train, partial_y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

걸리는 시간, 손실값, 정확도가 나오네

val_loss 테스트데이터의 손실값과 

val_accuracy 테스트데이터의 정확도 

In [ ]:
history_dict = history.history

In [ ]:
history_dict.items() #dict일 경우 전체 내용을 보려면 items 쓰면 됨

In [ ]:
history_dict.keys() # val_accuracy 가 가장 중요해

이제 히스토리의 key값들을 가지고 그래프를 그려보자

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(1, len(acc)+1)

In [ ]:
plt.plot(epochs, loss, 'bo', label='Training loss') # y축은 손실값에 따라서 움직일거야
plt.plot(epochs, val_loss, 'b-', label='Validation loss')

# 제목 달자
plt.title('Training and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

이런 그래프를 그려보면서 언제쯤 그만두면 되는가! 생각을 해봅시다아앙
validation loss 와 training loss를 보면 교차되는 지점이 바로 최적화가 이뤄지는 구간인데

지금  그래프를 보면 겹치는 구간이 아예 없고 반복할 수록 오히려 

격차가 점점 벌어지고 있음 그러면 가장 처음이 좋은거임

지금 책이랑 결과값이 다를텐데 

다르게 나오는 이유는...데이터가 바뀌어섴ㅋㅋㅋ 게다가 케라스 버전도 바뀌었음...

## 이미지

이미지부터는 코랩에서 돌리기 좀 힘들 수 있음 
아나콘다 깔고 내장 로컬로 돌려야함

m1 칩셋에 에어 보급형은 gpu가 7개 고급형은 8개 

5장 봐야 아이폰으로 사진찍어서 구별하는 걸 할 수 있어

---

#### 질문: 사람들이 하트를 그렸는데 이 그린 걸 하트라는 걸 이해하려면 어떻게 해야 하나요?

사용자의 그림을 피쳐로 놓고 영어로 하트: 라벨
사람들이 손으로 그린...데이터가 많이 필요하겠쥬?

인식 못한 거를 데이터 라벨링 하고 다시 매칭시켜줘야 해
라벨링 개수가 높을 수록 정확도는 떨어질거야 그래서 
라벨링 수를 최소로 놓고( 예를 들면 개와 고양이 처럼 명확하게 두 개로 구분)


---

model 만드는 거 까진 구글 괜춘
마소는 클라우드 연동이 좋음
aws는 운용이 좋음


---

호갱노노

두 달안에 실거래가를 공개해야 하기 때문에 데이터가 많아 계에엥속 나왕

---

네이버맵 이용해서 함 해봐도 재밌겠다. 